# Loma Prieta Earthquake, Earthquake Occurrence Statistics: Omori's Law 

**Last week we:**
- pandas DataFrames, indexing, and data cleaning.
- Load marine geophysical data (bathymetry and marine magnetic anomalies) from two oceanic ridges.
- Select data and drop rows with NaNs.
- Plot bathymetry data and evaluate spreading rate.
- Declare a function to detrend and filter magnetic anomalie data.
- Plot marine magnetic anomaly data and compare spreading rates.

**Our goals for today:**
- Load a Bay Area seismic catalog of January 01,1989 to December 31, 1995.
- Compute the distance and time interval between Loma Prieta quake and subsequant earthquakes to indentify aftershocks.
- Filter the aftershocks from the catalog and looked at their distribution.


## Setup

Run this cell as it is to setup your environment.

In [ ]:
import numpy as np
import pandas as pd
import datetime
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import cartopy.crs as ccrs
import cartopy.feature as cfeature

On October 17 at 5:15pm PDT (October 18 1989 at 04:15am UTC) the M6.9 Loma Prieta earthquake occurred in the Santa Cruz mountains approximately 80 km southwest of the Berkeley Campus. We will use this earthquake sequence to investigate the performance of catalog declustering algorithm.

https://en.wikipedia.org/wiki/1989_Loma_Prieta_earthquake

## Load the Earthquake Catalog

Load the .csv data file of all the earthquakes from January 01,1989 to December 31, 1995 in the ANSS (Advanced National Seismic System) catalog from between latitudes 36.0-38.5° and longitude -123.0 to -121.0° ([http://ncedc.org/anss/catalog-search.html](http://ncedc.org/anss/catalog-search.html)).

In [ ]:
# read data
LP_catalog=pd.read_csv('data/bay_area_anss_1989_1995.csv')
LP_catalog['DateTime'] = pd.xxx
LP_catalog.head()

DateTime objects are great!!

In [ ]:
LP_catalog['DateTime'].dt.year

In [ ]:
LP_catalog.iloc[2]['DateTime'].year

In [ ]:
#  create data arrays, it will speed up our loops later
year=
month=
day=
lat=
lon=
mag=
#number of events 
nevt=len(year)
print(nevt)

## Map the Raw Earthquake Catalog

On a map of the Bay Area plot the location of events in the raw catalog. Scale the marker color and size by magnitude.


In [ ]:
#Make a Map of the earthquake catalog

# Set Corners of Map
lat0=
lat1=
lon0=
lon1=
tickstep=0.5 #for axes
latticks=np.arange(lat0,lat1+tickstep,tickstep)
lonticks=np.arange(lon0,lon1+tickstep,tickstep)

# coordinates for UC Berkeley
Berk_lat = 37.8716
Berk_lon = -122.2727

plt.figure(1,(10,10))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([lon0, lon1, lat0, lat1], crs=ccrs.PlateCarree())
ax.coastlines(resolution='10m',linewidth=1)
ax.set_xticks(lonticks)
ax.set_yticks(latticks, crs=ccrs.PlateCarree())
ax.set(xlabel='Longitude', ylabel='Latitude',title='Raw Catalog')

# Sort by magnitude to plot largest events on top
LP_catalog_sorted = ...
#exponent to scale marker size
z=...

plt.scatter(..., ..., s=..., 
            c=..., cmap='plasma',alpha=0.4,marker='o') # plot circles on EQs
plt.plot(Berk_lon,Berk_lat,'s',color='limegreen',markersize=8)  # plot green square on Berkeley Campus
plt.colorbar(label='Magnitude')

plt.show()

## Plot Magnitude vs. Time for the Raw Catalog

Plot magnitude vs. time for the raw catalog and print out the number of events as we did in-class. Use the `alpha = 0.2` argument in `plot` to make the markers transparent. 

In [ ]:
# plot magnitude vs. time
fig, ax = plt.subplots(figsize=(6,6))

ax.plot(..., ...,'o',alpha=0.2,markersize=5)
locator = mdates.AutoDateLocator()
formatter = mdates.ConciseDateFormatter(locator)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)
ax.set(xlabel='Time', ylabel='Magnitude',title='Raw Event Catalog')
ax.grid()
plt.show()

## Identify the aftershocks

For each earthquake in the catalog with magnitude M, the subsequent earthquakes are determined to be aftershocks if they occur within a distance ($d$, in km) and time interval ($t$, in days). 

<img src="Figures/aftershock_windows.png" width=600>

To build our algorithm to identify aftershock using these windows we need to convert the year-month-day formate of dates to a timeline in number of days. We'll do this using the function `datetime.date` which for a given year, month, and day returns a datetime class object, which can be used to compute the time interval in days.

###  Determine the index value and number of days from the Loma Prieta Quake

Use the pandas dataframe index method `dataframe_name.index` to find where in the catalog the largest quake happens.

In [ ]:
LP_ind = ...
LP_ind

In [ ]:
#Determine the number of days from the Loma Prieta
days=np.zeros(nevt) # initialize the size of the array days

for i in range(0,nevt,1):
    d0 = datetime.date(..., ..., ...)
    d1 = datetime.date(..., ..., ...)
    delta = d1 - d0
    days[i]=delta.days # fill days in with the number of days since the first event

### Define a function to compute the distance between to geographic points on a sphere

We also need a function to compute the great circle distance in km between earthquakes. We'll use the haversine formula for the great circle distance which is well conditioned for small distances.

<img src="Figures/great_circle_eqn.png" >


<img src="Figures/Illustration_of_great-circle_distance.svg" >
Great-circle distance shown in red between two points on a sphere, P and Q. 
Source: https://en.wikipedia.org/wiki/Great-circle_distance

In [ ]:
#This function computes the spherical earth distance between to geographic points and is used in the
#declustering algorithm below
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.
    
    The first pair can be singular and the second an array

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2]) # convert degrees lat, lon to radians

    dlon = ...
    dlat = ...

    a = ...  # great circle inside sqrt

    c = ...   # great circle angular separation
    km = ...   # great circle distance in km, earth radius = 6371.0 km
    return km

An approximation of the windows sizes according to Gardner and Knopoff (1974) is shown in the equation below. Using this approximation makes programming a windowing algorithm easier. 

<img src="Figures/window_approx.png" >

Use `np.power(base, exponent)` to compute the distance and time interval bounds.

In [ ]:
Dtest=...   # distance bounds

if mag[i] >= ...:
    Ttest=...  # aftershock time bounds for M >= 6.5
else:
    Ttest=...  # aftershock time bounds for M < 6.5

### Put the pieces together to build a aftershock detection algorithm

In [ ]:
#Decluster the Catalog  Note: This cell may take a few minute to complete
cnt=... # initialize a counting variable
save=np.zeros((1,1000000),dtype=int) # initialize a counting variable

i=...
# logical if statements to incorporate definitions of Dtest and Ttest aftershock window bounds
Dtest=...   # distance bounds
if mag[i] >= ...:
    Ttest=...  # aftershock time bounds for M >= 6.5
else:
    Ttest=...  # aftershock time bounds for M < 6.5

a=...    # time interval in days to subsequent earthquakes in catalog
m=...   # magnitudes of subsequent earthquakes in catalog
b=haversine_np(lon[i],lat[i],lon[...],lat[...]) # distance in km to subsequent EQs in catalog

icnt=np.count_nonzero(a <= Ttest)   # counts the number of potential aftershocks, 
                                    # the number of intervals <= Ttest bound
if icnt > 0:  # if there are potential aftershocks
    itime=np.array(np.nonzero(a <= Ttest)) + (i+1) # indices of potential aftershocks <= Ttest bound
    for j in range(0,icnt,1):   # loops over the aftershocks         
        if b[j] <= Dtest and m[j] < mag[i]: # test if the event is inside the distance window 
                                            # and that the event is smaller than the main EQ
            save[0][cnt]=itime[0][j]  # index value of the aftershock
            cnt += 1 # increment the counting variable

                
af_ind=...   # This is an array of indexes that will be used to delete events flagged 
                                      # as aftershocks    

Create a set of arrays for the aftershock catalog. Use `af_ind` to select the aftershock events fomr the raw calalog.

In [ ]:
# select the aftershock events
aftershock_df = ...
aftershock_days=...  #The aftershocks are selected from the days array 
aftershock_mag=...    #The aftershocks are selected from the mag array 
aftershock_lon=...    #The aftershocks are selected from the lon array 
aftershock_lat=...   #The aftershocks are selected from the lat array 
n2=len(aftershock_days)

<font color=darkred>**_Concept question:_**</font> How many aftershock events were there?

**Write your answer here.**

## Plot Magnitude vs. Time for the Aftershock Catalog

Plot magnitude vs. time for the aftershock events and print out the number of events. Use the `alpha = 0.2` argument in `plot` to make the markers transparent. 

In [ ]:
#Plot DeClustered Catalog
fig, ax = plt.subplots(figsize=(6,6))

ax.plot(mdates.date2num(LP_catalog.iloc[LP_ind]['DateTime']), mag[LP_ind[0]],'o',
        color='red',markersize=10,label='Loma Prieta')
ax.plot(..., ...,'o',alpha=0.2,markersize=5,label='Aftershocks')

locator = mdates.AutoDateLocator()
formatter = mdates.ConciseDateFormatter(locator)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)

ax.set(xlabel='Time', ylabel='Magnitude',title='Aftershock Event Catalog')
ax.grid()
ax.legend()
plt.show()

## Plot a histogram of `aftershock_days`
Plot a histogram of the number of aftershocks per 20-day period after the main shock. Use `bins=range(0,920,20)`.

<font color=darkred>**_Discussion question:_**</font> How would you describe the distribution of number of aftershocks with time after the main quake?

**Write your answer here.**

## Map the Aftershock Events

On a map of the Bay Area plot the location of events in the aftershock catalog. Again scale the marker color and size by magnitude, set `vmax=6.9` so the colorscale matches our original map. Add the locations of UC Berkeley campus and the Loma Prieta event epicenter. 

In [ ]:
#Make a Map of Aftershock events



<img src="Figures/fault_map.png" width=700>
Map of Bay Area faults. 
Source: https://pubs.er.usgs.gov/publication/fs20163020

<font color=darkred>**_Discussion questions:_**</font>  What faults were active? Were aftershocks only triggered on the same fault as the main quake?

**Write your answer here.**

What are the probabilities of one or more M$\geq$6.7 quake on these fault in the next 23 years?  

**Write your answer here.**

### Turn in this notebook

Save your completed notebook. Click on __File, Download as, HTML__ to save the notebook as a HTML file. Upload it to the [bCourses assignment page](https://bcourses.berkeley.edu/courses/1498475/assignments).